# Comparison of temperature and salinity sections along the OVIDE campaign

 - the coordinates of the OVIDE section : coord_OVIDE.txt
 - date of the campaign : 23/06/2012 to 20/07/2012
 - the OVIDE data are in : /scratch/cnt0024/hmg2840/albert7a/OVIDE/
 

In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_sections',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 
cluster.scale(420)
cluster

from dask.distributed import Client
client = Client(cluster)
client



/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


Client Scheduler: tcp://172.30.100.4:34922 Dashboard: http://172.30.100.4:43710/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [2]:
!squeue -u albert7a

import time
nb_workers = 0
while True:
    nb_workers = len(client.scheduler_info()["workers"])
    if nb_workers >= 2:
        break
    time.sleep(1)
print(nb_workers)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           9789590     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789591     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789592     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789593     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789594     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789595     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789596     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789597     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789598     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789599     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789600     hsw24 make_sec albert7a PD       0:00      1 (Priority)
           9789601     hsw24 make_sec albert7a

In [3]:
import xarray as xr

import glob
import os

import numpy as np
import numpy.matlib
import numpy.ma as ma

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import pyproj
from pyproj import Geod
geod = Geod(ellps='WGS84')

%matplotlib inline

## Define the OVIDE domain in eNATL60 data : in notebook sections-OVIDE/2020-03-04-AA-domain-OVIDE-in-eNATL60.ipynb

Results : 
 - imin = 2916
 - imax = 5296
 - jmin = 2257
 - jmax = 4066

In [4]:
## Dataset

gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/coordinates_eNATL60.nc'
maskfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6.nc'


In [5]:
dsgrid=xr.open_dataset(gridfile)
nav_lon=dsgrid.nav_lon
nav_lat=dsgrid.nav_lat

In [6]:
coord_secOV_file='/scratch/cnt0024/hmg2840/albert7a/OVIDE/coord_section_OVIDE2012-2.nc'
dsOV=xr.open_dataset(coord_secOV_file)
latOV=dsOV.latitude
lonOV=dsOV.longitude

In [7]:
jmin = 2257
jmax = 4066
imin = 2916
imax = 5296


In [8]:
nav_lon_OVIDE=nav_lon[jmin:jmax,imin:imax]
nav_lat_OVIDE=nav_lat[jmin:jmax,imin:imax]

# Extract eNATL60 OVIDE domain and period

In [9]:
dir='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02*-S/'
tfiles06='*/eNATL60-BLBT02*_gridT_201006??-201006??.nc'
tfiles07='*/eNATL60-BLBT02*_gridT_201007??-201007??.nc'

list_tfiles06=sorted(glob.glob(dir+tfiles06))
list_tfiles07=sorted(glob.glob(dir+tfiles07))

list_tfiles06_07=list_tfiles06+list_tfiles07

sfiles06='*/eNATL60-BLBT02*_gridS_201006??-201006??.nc'
sfiles07='*/eNATL60-BLBT02*_gridS_201007??-201007??.nc'

list_sfiles06=sorted(glob.glob(dir+sfiles06))
list_sfiles07=sorted(glob.glob(dir+sfiles07))

list_sfiles06_07=list_sfiles06+list_sfiles07


In [10]:
dst=xr.open_mfdataset(list_tfiles06_07,chunks={'x':700,'y':700,'deptht':1})
dstsel=dst.sel(time_counter=slice('2010-06-23','2010-07-20'))
temp_mean_OVIDE=dstsel.votemper[:,:,jmin:jmax,imin:imax].mean(dim='time_counter')


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/backends/api.py:933: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [11]:
dss=xr.open_mfdataset(list_sfiles06_07,chunks={'x':700,'y':700,'deptht':1})
dsssel=dss.sel(time_counter=slice('2010-06-23','2010-07-20'))
salt_mean_OVIDE=dsssel.vosaline[:,:,jmin:jmax,imin:imax].mean(dim='time_counter')


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/backends/api.py:933: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


In [12]:
def section_from_debut_fin_points(lon_ar7w_debut,lat_ar7w_debut,lon_ar7w_fin,lat_ar7w_fin,navlon,navlat,var):
    
    ee = pyproj.Geod(ellps='GRS80') # ellipsoid used for distance calculation
    
    def distance(lat1,lon1,lat2,lon2,geoid):
        az12,az21,dist = geoid.inv(lon1,lat1,lon2,lat2)
        return dist

    def loclatlon(navlon,navlat,lon,lat):
        ind_debut=np.where((navlon>lon-0.1)&(navlon<lon+0.1)&(navlat>lat-0.1)&(navlat<lat+0.1))
        dis=np.empty(len(ind_debut[0]))
        for ii in range(0,len(ind_debut[0])):
            dis[ii]=np.abs(distance(lat,lon,navlat[ind_debut[0][ii],ind_debut[1][ii]],navlon[ind_debut[0][ii],ind_debut[1][ii]],ee))
        ind=np.argmin(dis)
        i_debut=ind_debut[0][ind]
        j_debut=ind_debut[1][ind]
    
        return i_debut,j_debut
 
    def secfromdebuttofin(i_debut,j_debut,i_fin,j_fin):
        off=1
        fromi=np.atleast_1d(i_debut)+off
        fromj=np.atleast_1d(j_debut)+off
        toi=np.atleast_1d(i_fin)+off
        toj=np.atleast_1d(j_fin)+off

        veci=fromi
        vecj=fromj

        a=(toj-fromj)/((toi-fromi).astype(np.float))
        b=toj-toi*a

        while not((veci[-1]==toi) & (vecj[-1]==toj)):
            newi=veci[-1]+np.sign(toi-fromi)
            newj=vecj[-1]+np.sign(toj-fromj)
            y=a*newi+b
            x=(newj-b)/(a.astype(np.float))

            if ((np.abs(x-veci[-1]))>=(np.abs(y-vecj[-1]))):
                veci=np.append(veci,newi)
                vecj=np.append(vecj,vecj[-1])
            else:
                veci=np.append(veci,veci[-1])
                vecj=np.append(vecj,newj)

        veci=veci.astype(np.int)-off
        vecj=vecj.astype(np.int)-off

        dis=np.empty(len(veci))
        for ii in range(0,len(veci)):
            dis[ii]=np.abs(distance(navlat[veci[0],vecj[0]],navlon[veci[0],vecj[0]],navlat[veci[ii],vecj[ii]],navlon[veci[ii],vecj[ii]],ee))
    
        return veci,vecj,dis
            
    i_debut,j_debut=loclatlon(navlon,navlat,lon_ar7w_debut,lat_ar7w_debut)
    i_fin,j_fin=loclatlon(navlon,navlat,lon_ar7w_fin,lat_ar7w_fin)

    veci,vecj,dis=secfromdebuttofin(i_debut,j_debut,i_fin,j_fin)

    if np.ndim(var) == 4:
        [nt,nk,nj,ni]=var.shape
        var_sec=np.empty([nt,nk,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,:,ii]=var[:,:,veci[ii],vecj[ii]]
    if np.ndim(var) == 3:
        [nt,nj,ni]=var.shape
        var_sec=np.empty([nt,len(veci)])
        for ii in range(0,len(veci)):
            var_sec[:,ii]=var[:,veci[ii],vecj[ii]]
    
    return veci,vecj,dis,var_sec

In [13]:
temp_mean_OVIDE.shape

(300, 1809, 2380)

In [14]:
lon_debut=lonOV[0]
lat_debut=latOV[0]
lon_fin=lonOV[1]
lat_fin=latOV[1]
veci1,vecj1,dis1,temp1=section_from_debut_fin_points(lon_debut,lat_debut,lon_fin,lat_fin,nav_lon_OVIDE,nav_lat_OVIDE,temp_mean_OVIDE)

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in true_divide
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in add
/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide
distributed.client - WARNING - Couldn't gather 2 keys, rescheduling {"('getitem-d68e3121f38c5b52e1d50123243944b6', 195)": (), "('getitem-d68e3121f38c5b52e1d50123243944b6', 219)": ()}
distributed.client - WARNING - Couldn't gather 5 keys, rescheduling {"('getitem-d68e3121f38c5b52e1d50123243944b6', 235)": (), "('getitem-d68e3121f38c5b52e1d50123243944b6', 292)": (), "('getitem-d68e3121f38c5b52e1d50123243944b6', 238)": (), "('getitem-d68e3121f38c5b52e1d50123243944b6', 232)": (), "('getitem-d68e3121f38c5b52e1d50123243944b6', 244)": ()}
distr

KilledWorker: ("('concatenate-open_dataset-getitem-23b5728ea513aba1bd51af337c0c167c', 11, 277, 0, 3)", <Worker 'tcp://172.30.8.254:33066', memory: 0, processing: 8>)